In [1]:
import gzip
from collections import defaultdict
from datetime import datetime

In [2]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [3]:
dataset_path = '../../dataset/'
dataset_name = 'Baby_5'

In [4]:
# 筛时间
T = 201404

# 保证用户在截断时间之前有交互
countUafterT = defaultdict(int)

for l in parse(dataset_path + 'reviews_'+ dataset_name +'.json.gz'):
    asin = l['asin']
    rev = l['reviewerID']
    time = datetime.fromtimestamp(l['unixReviewTime']).__format__('%Y%m%d')
    
    if int(time[0:6]) >= T:
        countUafterT[rev] += 1

In [5]:
# 筛时间
# 保证用户在截断时间之前与之后有交互
countUbeforeT = defaultdict(int)
# 保证物品在截断时间之前有交互
countIbeforeT = defaultdict(int)

for l in parse(dataset_path + 'reviews_'+ dataset_name +'.json.gz'):
    asin = l['asin']
    rev = l['reviewerID']
    time = datetime.fromtimestamp(l['unixReviewTime']).__format__('%Y%m%d')
    
    if rev not in countUafterT.keys(): continue
    
    if int(time[0:6]) < T:
        countIbeforeT[asin] += 1
        countUbeforeT[rev] += 1

In [6]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0

User = dict()
Item = dict()

for l in parse(dataset_path + 'reviews_'+ dataset_name +'.json.gz'):
    asin = l['asin']
    rev = l['reviewerID']
    time = datetime.fromtimestamp(l['unixReviewTime']).__format__('%Y%m%d')
    rating = l['overall']

    if rev not in countUbeforeT.keys() or asin not in countIbeforeT.keys(): continue
    
    if rev in usermap: userid = usermap[rev]
    else:
        userid = usernum
        usernum += 1
        usermap[rev] = userid
        User[userid] = []
    
    if asin in itemmap: itemid = itemmap[asin]
    else:
        itemid = itemnum
        itemnum += 1
        itemmap[asin] = itemid
        Item[itemid] = []
    
    
    User[userid].append([itemid, time, rating])
    Item[itemid].append([userid, time, rating])

In [7]:
usernum

4314

In [8]:
itemnum

5854

In [9]:
num_int = 0
for k,v in User.items():
    num_int += len(v)

In [10]:
num_int

42657

In [11]:
num_int / (usernum * itemnum) * 100

0.16891081214513762

In [12]:
# user部分
User_sorted = dict()
for k,v in User.items():
    User_sorted[k] = sorted(v, key=lambda v: v[1])

In [13]:
User_train = defaultdict(list)
User_test = defaultdict(list)

for k,v in User_sorted.items():
    for i in v:
        time = i[1]
        if int(time[0:6]) < T:
            User_train[k].append(i)
        else:
            User_test[k].append(i)

In [14]:
len(User_train)

4314

In [15]:
len(User_test)

4046

In [16]:
num_tr = 0
for k,v in User_train.items():
    num_tr += len(v)
num_tr

31567

In [17]:
num_ts = 0
for k,v in User_test.items():
    num_ts += len(v)
num_ts

11090

In [18]:
# 训练集测试集占比
num_ts / (num_tr + num_ts)

0.25998077689476523

In [19]:
# 保证测试集里的所有物品都在训练集里出现过

In [20]:
setI = set()
for k,v in User_train.items():
    for i in v:
        setI.add(i[0])

In [21]:
len(setI)

5854

In [22]:
itemnum

5854

In [23]:
len(setI) == itemnum

True

In [24]:
path = './baby/train.txt'
f = open(path, 'w')

for u, i in User_train.items():
    f.write(str(u))
    for n in i:
        f.write(" " + str(n[0]))
    f.write('\n')

f.close()

In [25]:
path = './baby/test.txt'
f = open(path, 'w')

for u, i in User_test.items():
    f.write(str(u))
    for n in i:
        f.write(" " + str(n[0]))
    f.write('\n')

f.close()